# TF acs-2 screen

In [2]:
%matplotlib inline
import os
import sys
import tifffile as tiff
import pandas as pd
import matplotlib
# For visualising
#matplotlib.use('MacOSX') 
# For saving
#matplotlib.use('Agg') 
matplotlib.use('MacOSX') #TkAgg
import matplotlib.pyplot as plt

import numpy as np
import itertools as IT
import time
import glob
import skimage
from skimage import io as skio
from skimage import color

import cPickle
import gzip

import datetime
from multiprocessing import Pool, Manager
from multiprocessing.sharedctypes import Value, Array
from ctypes import Structure, c_double, c_int

sys.path.append("./src")
from utilities import *

#plt.interactive(True)
matplotlib.rcParams.update({'font.size': 12})
np.set_printoptions(precision=3)



## Set working directory

In [3]:
os.chdir("/Users/Povilas/Dropbox/Projects/Metformin_TF_acs-2/")
sourceloc = "/Users/Povilas/Dropbox/Projects/Metformin_TF_acs-2/"

# os.chdir("/home/pnorv/Dropbox/Projects/Metformin_TF_acs-2/")
# sourceloc = "home/pnorv/Dropbox/Projects/Metformin_TF_acs-2/"

odir="."

## Collect info about files

In [5]:
filedata=[]
header=['Replicate_folder','Folder','File']
filedata.append(header)

allfiles=glob.glob('./Rep*/*/*.tiff')

for fl in allfiles:
    rep,fld,fln=fl.split('/')[1:]
    filedata.append([rep,fld,fln])
    #print rep, fld, fln
    
writecsv(filedata,'Allfiles_additional.csv')

## Parse info

In [6]:
#filelist=pd.read_csv('./Allfiles_annotated.csv')

oimages="Images_additional.pkl"

#oimagesgz="Images.pkl.gz"

In [9]:
filesel=allfiles

## Collect images

In [10]:
settings={'colordepth':'uint8'}
args=[(fl,settings,) for fl in filesel]

results=multiprocessor(4,collector_M,args)
fls,imagesarray=zip(*results)

imagesindex_add={fl:fli for fli,fl in enumerate(fls) }

print("Saving images: {}".format(time.ctime()))
print "Stacking..."
images_add=np.stack(imagesarray, axis=0)
print "Done!"

f = open(oimages, "w")
#f = gzip.open(oimagesgz, "w")
print "Writing..."
cPickle.dump((imagesindex_add,images_add), f)
f.close()
os.system("gzip {}".format(oimages))
print("Saving images complete!")



Done!
Saving images: Tue Mar 13 14:06:35 2018
Stacking...
Done!
Writing...
Saving images complete!


In [48]:
f = open("Images_all.pkl", "w")
#f = gzip.open(oimagesgz, "w")
print "Writing..."
cPickle.dump((imagesindex,images_joint), f)
f.close()

os.system("gzip {}".format("Images_all.pkl"))
print("Saving images complete!")

Writing...
Saving images complete!


## Load images

In [4]:
iimagesgz="Images_all.pkl.gz"
print time.ctime()
print "Loading images... {}".format(time.ctime())
idata = gzip.open(iimagesgz,'rb')
imagesindex, images = cPickle.load(idata)
idata.close()
print "Loading finished! {}".format(time.ctime())
print time.ctime()
max(imagesindex.values())

Tue Mar 13 16:10:17 2018
Loading images... Tue Mar 13 16:10:17 2018
Loading finished! Tue Mar 13 16:11:45 2018
Tue Mar 13 16:11:45 2018


930

## Save heatmaps

In [55]:
filelist=pd.read_csv('./Allfiles_annotated.csv')

for index,row in filelist.iterrows():
    if row['Type']=='C':
        # and row['Replicate']==1 and row['Gene']=='arcA'
        dataset=row['Dataset']
        rep=row['Replicate']
        tp=row['Type']
        repf=row['Replicate_folder']
        fold=row['Folder']
        fin=row['FileNo']
        fl=row['File']
        fpat="./{}/{}/{}".format(repf,fold,fl)
        
        hrepf="{}/Rep{}/{}".format(dataset,rep,fold)
        
        if not os.path.exists(hrepf):
            os.makedirs(hrepf)
        
        fheat="./{}/Rep{}/{}/{} TF{}_H.tiff".format(dataset,rep,fold,fold,fin)
        
        #print rep, tp, fpat, fheat
        image=images[imagesindex[fpat]]
        #dv = rank.median(image, disk(3))
        #Expanding 
        #comb = opening(dv, selem=disk(od))
        
        imgjet=jetimage(image)
        skio.imsave(fheat,imgjet)
        
    

/usr/local/lib/python2.7/site-packages/skimage/exposure/exposure.py:307: RuntimeWarning: invalid value encountered in true_divide
  image = (image - imin) / float(imax - imin)
/usr/local/lib/python2.7/site-packages/skimage/util/dtype.py:110: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))
/usr/local/lib/python2.7/site-packages/skimage/io/_io.py:132: UserWarning: ./Original/Rep1/mlc wo metf/mlc wo metf TF1_H.tiff is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/site-packages/skimage/io/_io.py:132: UserWarning: ./Original/Rep1/csiR wo metf/csiR wo metf TF1_H.tiff is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/site-packages/skimage/io/_io.py:132: UserWarning: ./Original/Rep1/ntrC wo metf/ntrC wo metf TF1_H.tiff is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/site-packages/skimage/io/_io.

## Segment images

In [47]:
olabels="Segmentation_onlyprc_additional.pkl"
otbl="All_results_onlyprc_additional.csv"

In [48]:
imagesindexsel={ik:ival for ik, ival in imagesindex.items() if "additional" in ik }

In [50]:
#settings={'fd':10}
settings={'pmax':4,'fd':10}

args=[(fln,images[imagesindexsel[fln]],settings) for fln in imagesindexsel.keys()]

results=multiprocessor(4,labeller4_M,args)

fls, filteredarray, labelsarray, resultsarray = zip(*results)

#Collect and save results
labelsindex={fl:fli for fli,fl in enumerate(fls) }

results_data=[]
res_header=['File','Worm','W_N','W_Sum','W_Mean','B_N','B_Sum','B_Mean']

for fl, respack in zip(fls, resultsarray):
    for row in respack:
        if len(row)>0:
            results_data.append([fl]+row)
            

# Save results
results=pd.DataFrame(results_data,columns=res_header)
results.to_csv(otbl)

print("Saving labels: {}".format(time.ctime()))
print "Stacking..."

labels=np.stack(labelsarray, axis=0)
filtered=np.stack(filteredarray, axis=0)
print "Stacking complete!"

print "Writing..."
#f = gzip.open(olabelsgz, "w")
f = open(olabels, "w")
cPickle.dump((labelsindex,labels), f)
#cPickle.dump((labelsindex,filtered,labels), f)
f.close()
print "Zipping..."
os.system("gzip -f {}".format(olabels))
print("Saving labels complete! {}".format(time.ctime()))

/usr/local/lib/python2.7/site-packages/skimage/util/dtype.py:110: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))
/usr/local/lib/python2.7/site-packages/skimage/util/dtype.py:110: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))
/usr/local/lib/python2.7/site-packages/skimage/util/dtype.py:110: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))
/usr/local/lib/python2.7/site-packages/skimage/util/dtype.py:110: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))
/usr/local/lib/python2.7/site-packages/skimage/util/dtype.py:106: UserWarning: Possible sign loss when converting negative image of type int32 to positive image of type uint8.
  "%s to positive image of type %s." % (dtypeobj_in, dtypeobj))
/usr/local/lib/python2.7/site-packag

Done   2%,     4/196   remaining: 0:08:02 total: 0:08:12


/usr/local/lib/python2.7/site-packages/skimage/util/dtype.py:136: UserWarning: Downcasting int32 to uint8 without scaling because max value 1 fits in uint8
  warn(msg)
/usr/local/lib/python2.7/site-packages/skimage/util/dtype.py:136: UserWarning: Downcasting int32 to uint8 without scaling because max value 0 fits in uint8
  warn(msg)
/usr/local/lib/python2.7/site-packages/skimage/util/dtype.py:136: UserWarning: Downcasting int32 to uint8 without scaling because max value 1 fits in uint8
  warn(msg)


Done   4%,     8/196   remaining: 0:05:53 total: 0:06:08
Done   6%,    12/196   remaining: 0:05:07 total: 0:05:27
Done   8%,    16/196   remaining: 0:04:41 total: 0:05:06


/usr/local/lib/python2.7/site-packages/skimage/util/dtype.py:136: UserWarning: Downcasting int32 to uint8 without scaling because max value 3 fits in uint8
  warn(msg)


...

Done  90%,   178/196   remaining: 0:00:24 total: 0:04:30
Done  92%,   181/196   remaining: 0:00:20 total: 0:04:30


/usr/local/lib/python2.7/site-packages/skimage/util/dtype.py:136: UserWarning: Downcasting int32 to uint8 without scaling because max value 2 fits in uint8
  warn(msg)


Done  94%,   186/196   remaining: 0:00:13 total: 0:04:28
Done  96%,   189/196   remaining: 0:00:09 total: 0:04:29
Done  97%,   192/196   remaining: 0:00:05 total: 0:04:30


/usr/local/lib/python2.7/site-packages/skimage/util/dtype.py:136: UserWarning: Downcasting int32 to uint8 without scaling because max value 2 fits in uint8
  warn(msg)


Done  99%,   195/196   remaining: 0:00:01 total: 0:04:31
Done!
Saving labels: Tue Mar 13 18:13:04 2018
Stacking...
Stacking complete!
Writing...
Zipping...
Saving labels complete! Tue Mar 13 18:13:40 2018


### Load data

In [4]:
fileinfo=readTF('./Allfiles_annotated.csv')
filelist=pd.read_csv('./Allfiles_annotated.csv')
genes=list(set(fileinfo['Original'].keys()+fileinfo['Additional'].keys()))
#genes=['OP50-C']

iimages="Images_all.pkl.gz"
ilabels="Segmentation_onlyprc_additional.pkl.gz"

ofigs="Segmentation_onlyprc_add/{}_{}_segmentation_onlyprc_add.pdf"

datasets=["Original","Additional"]

repflds=['Rep1_23-8-17','Rep2_24-8-17']

plt.ioff()

./Allfiles_annotated.csv
Index([u'Dataset', u'Replicate', u'Type', u'Gene', u'FileNo', u'FileInd',
       u'Replicate_folder', u'Folder', u'File', u'Max'],
      dtype='object')
Necessary headers found!


In [5]:
print time.ctime()

print "Loading images..."
imagesf = gzip.open(iimages,'rb')
imagesindex, images = cPickle.load(imagesf)
imagesf.close()
print "Loading finished!"
print time.ctime()

print "Loading labels..."
labelsf = gzip.open(ilabels,'rb')
labelspickle = cPickle.load(labelsf)
if len(labelspickle)==3:
    labelsindex, filtered, labels=labelspickle
else:
    labelsindex, labels=labelspickle
labelsf.close()
print "Loading finished!"
print time.ctime()

Wed Mar 14 11:07:10 2018
Loading images...
Loading finished!
Wed Mar 14 11:08:28 2018
Loading labels...
Loading finished!
Wed Mar 14 11:08:35 2018


In [7]:
fileinfo['Additional'].keys()

['OP50-C', 'nac', 'marA']

### Generate figures

In [11]:
#def plotTF_M((gene,ofigs,fileinfo,imagesindex,images,labelsindex,filtered,labels, settings,q)):
    #ofigs,fileinfo, imagesindex, labelsindex, filtered, labels,
    #global
    
#     fileinfo=Afileinfo
#     imagesindex=Aimagesindex
#     images=Aimages
#     labelsindex=Alabelsindex
#     filtered=Afiltered
#     labels=Alabels

fd=10
od=5

for dataset in ["Additional"]: #"Original",
    genes=fileinfo[dataset].keys()
    for gene in genes:   
        if gene=='OP50-C' and dataset=="Original":
            nflds=6*2
        else:
            nflds=4*2
        print dataset, gene
        
        gval=fileinfo[dataset][gene]
        gmax = gval['Max']
        print gmax

        fwidth=6*nflds
        fheight=5*gmax

        #print nflds, gmax, fwidth, fheight
        fig, axes = plt.subplots(nrows=gmax, ncols=nflds, figsize=(fwidth,fheight), dpi=300)
        ofile=ofigs.format(dataset,gene)

        #print ofile
        col=0
        for rep in [1,2]:
            repval = gval[rep]
            
            if dataset=="Original":
                repfld=repflds[rep-1]
            else:
                repfld="Rep{}_additional".format(rep)
            
            
            for tp in ['C','T']:
                tpval=repval[tp]
                for fld, fldvals in tpval.iteritems():
                    print "Gene: {} Replicate: {} Type: {} Folder: {}".format(gene, rep, tp, fld)
                    fldcontsel=fldvals['ByFIN']
                    #print fldcontsel.keys()

                    row=0
                    for fli in fldcontsel.keys():
                        fl=fldcontsel[fli]
                        
                        flfull="./{}/{}/{}".format(repfld,fld,fl)
                        print flfull #, os.path.isfile(flfull)

                        #GetRGB data from a collapsed image
                        image_raw=images[imagesindex[flfull]]
                        if np.ndim(image_raw)==2:
                            #v=image_raw
                            image=color.gray2rgb(image_raw)
                        else:
                            #v=color.rgb2hsv(image)[:,:,2]
                            image=image_raw
                        #Load labels
                        
                        dv = rank.median(image_raw, disk(fd))
                        #Expanding 
                        comb = opening(dv, selem=disk(od))

                        imagef=skimage.img_as_ubyte(comb)#filtered[labelsindex[flfull]]
                        
                        labeled_worms=labels[labelsindex[flfull]]

                        plotcontours(imagef,axes[row,col],ylabel=fl,title="{} - {}".format(rep,fld),plotcontour=False,plotlabels=False,white=False)
                        plotcontours(image,axes[row,col+1],labeled_worms,title="{}".format(fli),plotlabels=False)
                        row+=1 

                        #q.put(flfull) 
                    col+=2

        #fig.tight_layout()
        fig.savefig(ofile, bbox_inches='tight')
        plt.close(fig)

print "Finished!"

Additional OP50-C
18
Gene: OP50-C Replicate: 1 Type: C Folder: op50-c wo metf
./Rep1_additional/op50-c wo metf/op50-c wo metf1.tiff


/usr/local/lib/python2.7/site-packages/matplotlib/artist.py:210: MatplotlibDeprecationWarning: This has been deprecated in mpl 1.5, please use the
axes property.  A removal date has not been set.
  warnings.warn(_get_axes_msg, mplDeprecation, stacklevel=1)


./Rep1_additional/op50-c wo metf/op50-c wo metf2.tiff
./Rep1_additional/op50-c wo metf/op50-c wo metf3.tiff
./Rep1_additional/op50-c wo metf/op50-c wo metf4.tiff
./Rep1_additional/op50-c wo metf/op50-c wo metf5.tiff
./Rep1_additional/op50-c wo metf/op50-c wo metf6.tiff
./Rep1_additional/op50-c wo metf/op50-c wo metf7.tiff
./Rep1_additional/op50-c wo metf/op50-c wo metf8.tiff
./Rep1_additional/op50-c wo metf/op50-c wo metf9.tiff
./Rep1_additional/op50-c wo metf/op50-c wo metf10.tiff
./Rep1_additional/op50-c wo metf/op50-c wo metf11.tiff
./Rep1_additional/op50-c wo metf/op50-c wo metf12.tiff
./Rep1_additional/op50-c wo metf/op50-c wo metf13.tiff
./Rep1_additional/op50-c wo metf/op50-c wo metf14.tiff
./Rep1_additional/op50-c wo metf/op50-c wo metf15.tiff
./Rep1_additional/op50-c wo metf/op50-c wo metf16.tiff
Gene: OP50-C Replicate: 1 Type: T Folder: op50-c metf
./Rep1_additional/op50-c metf/op50-c metf1.tiff
./Rep1_additional/op50-c metf/op50-c metf2.tiff
./Rep1_additional/op50-c metf/op5

## Multiprocessing figures - does not work

In [22]:
Aimages=Array(c_double,images.shape)
Aimages=images.copy()

Alabels=Array(c_double,labels.shape)
Alabels=labels.copy()

Afiltered=Array(c_double,filtered.shape)
Afiltered=filtered.copy()

m = Manager()
Afileinfo=m.dict()
Afileinfo.update(fileinfo)

Aimagesindex=m.dict()
Aimagesindex.update(imagesindex)

Alabelsindex=m.dict()
Alabelsindex.update(labelsindex)

settings={}

#global fileinfo, imagesindex, images, labelsindex,filtered, labels

#global Afileinfo,Aimagesindex,Aimages,Alabelsindex,Afiltered,Alabels

args=[(gene,ofigs,Afileinfo,Aimagesindex,Aimages,Alabelsindex,Afiltered,Alabels, settings) for gene in genes]

results=multiprocessor(4,plotTF_M,args,filelist.shape[0])
